In [1]:
# 上下文语境

In [2]:
# 通过增加特征提取函数，我们可以修改词性标注器来利用各种词内部的其他特征，如：词长、它所包含的音节数或者它的前缀。
# 特征提取器仅仅只看目标词，我们没法添加依赖词出现的上下文语境特征。
# 然而上下文语境特征往往提供关于正确标记的强大线索。
# 如：a fly, fly前面的词是a可以确定fly是一个名词，而不是动词。

In [3]:
# 为了采取基于词的上下文的特征，我们必须修改以前为我们的特征提取器的定义的模式。
# 不是只传递已标注的词，我们将传递整个（未标注的）句子，以及目标词的索引。

In [6]:
import nltk
from nltk.corpus import brown

In [7]:
def pos_features(sentence,i):
    '''
    词性分类器，它的特征检测器检查一个词出现的上下文以便决定应该分配的词性标记。
    特别的，前面的词被作为一个特征。
    '''
    features = {'suffix(1)':sentence[i][-1:],
               'suffix(2)':sentence[i][-2:],
               'suffix(3)':sentence[i][-3:]}
    if i == 0:
        features['prev-word'] = '<START>'
    else:
        features['prev-word'] = sentence[i-1]
    return features

In [8]:
pos_features(brown.sents()[0],8)  # 获取sentence的第8个词的3个后缀特征，加上第7个词的值。

{'prev-word': 'an', 'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion'}

In [21]:
tagged_sents = brown.tagged_sents(categories='news')

In [26]:
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i,(word,tag) in enumerate(tagged_sent):
        featuresets.append((pos_features(untagged_sent, i),tag))

In [28]:
featuresets

[({'prev-word': '<START>',
   'suffix(1)': 'e',
   'suffix(2)': 'he',
   'suffix(3)': 'The'},
  'AT'),
 ({'prev-word': 'The',
   'suffix(1)': 'n',
   'suffix(2)': 'on',
   'suffix(3)': 'ton'},
  'NP-TL'),
 ({'prev-word': 'Fulton',
   'suffix(1)': 'y',
   'suffix(2)': 'ty',
   'suffix(3)': 'nty'},
  'NN-TL'),
 ({'prev-word': 'County',
   'suffix(1)': 'd',
   'suffix(2)': 'nd',
   'suffix(3)': 'and'},
  'JJ-TL'),
 ({'prev-word': 'Grand',
   'suffix(1)': 'y',
   'suffix(2)': 'ry',
   'suffix(3)': 'ury'},
  'NN-TL'),
 ({'prev-word': 'Jury',
   'suffix(1)': 'd',
   'suffix(2)': 'id',
   'suffix(3)': 'aid'},
  'VBD'),
 ({'prev-word': 'said',
   'suffix(1)': 'y',
   'suffix(2)': 'ay',
   'suffix(3)': 'day'},
  'NR'),
 ({'prev-word': 'Friday',
   'suffix(1)': 'n',
   'suffix(2)': 'an',
   'suffix(3)': 'an'},
  'AT'),
 ({'prev-word': 'an', 'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion'},
  'NN'),
 ({'prev-word': 'investigation',
   'suffix(1)': 'f',
   'suffix(2)': 'of',
   'suffix(3)'

In [29]:
size = int(len(featuresets)*0.1)
train_set, test_set = featuresets[size:], featuresets[:size]  # 获取训练集和测试集

In [30]:
# 训练
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [31]:
# 计算准确度
nltk.classify.accuracy(classifier, test_set)

0.7891596220785678

In [ ]:
# 利用上下文特征相对于DecisionTree中的效果，很明显的提升，但是算法使用的不同，不能直接这样比较。
# 这边主要是了解到了上下文的特征的获取方式。